In [1]:
import numpy as np

In [2]:
npzfile = np.load('CPDM_results.npz')
droplets = npzfile['droplets']
test_batch = droplets[0:10000,...]

In [3]:
base_relative_abundances = [1e-4, 1e-3, 1e-2]

relative_abundances = [relative_abundance * number
                       for relative_abundance 
                       in base_relative_abundances
                       for number in (1,2,5) 
                       for repeat in range(10)]

relative_abundances += [1-sum(relative_abundances)]
frequencies = np.array(relative_abundances)

In [4]:
strain1 = 69
strain2 = 72

In [5]:
test = np.outer(np.exp(-frequencies), np.exp(-frequencies))

In [6]:
test

array([[0.99980002, 0.99980002, 0.99980002, ..., 0.95113431, 0.95113431,
        0.89395486],
       [0.99980002, 0.99980002, 0.99980002, ..., 0.95113431, 0.95113431,
        0.89395486],
       [0.99980002, 0.99980002, 0.99980002, ..., 0.95113431, 0.95113431,
        0.89395486],
       ...,
       [0.95113431, 0.95113431, 0.95113431, ..., 0.90483742, 0.90483742,
        0.8504412 ],
       [0.95113431, 0.95113431, 0.95113431, ..., 0.90483742, 0.90483742,
        0.8504412 ],
       [0.89395486, 0.89395486, 0.89395486, ..., 0.8504412 , 0.8504412 ,
        0.79931513]])

In [7]:
test[69,72]

0.9704455335485082

In [8]:
np.exp(-(frequencies[strain1] + frequencies[strain2]))

0.9704455335485082

In [9]:
single_strain_0_probs = np.exp(-frequencies)
double_strain_0_probs = np.outer(single_strain_0_probs, single_strain_0_probs)
double_strain_0_probs[strain1, strain2]

0.9704455335485082

In [10]:
np.sqrt(double_strain_0_probs[strain1, strain1])

0.9900498337491681

In [11]:
single_strain_0_probs[strain1]

0.9900498337491681

In [12]:
single_strain_non0_probs = 1. - single_strain_0_probs
double_strain_non0_probs = 1. - double_strain_0_probs

In [13]:
double_strain_non0_probs[strain1, strain2]

0.029554466451491845

In [14]:
conditional_strain1_non0_strain2_0_prob = 1. - (single_strain_non0_probs[strain2]/double_strain_non0_probs[strain1, strain2])

In [15]:
conditional_strain1_non0_strain2_0_prob

0.33000561097102066

In [16]:
conditional_strain2_non0_strain1_0_prob = 1. - (single_strain_non0_probs[strain1]/double_strain_non0_probs[strain1, strain2])

In [17]:
conditional_strain2_non0_strain1_0_prob

0.6633278334710173

In [18]:
1. - conditional_strain1_non0_strain2_0_prob - conditional_strain2_non0_strain1_0_prob

0.006666555557962028

hm, is that probability really supposed to be _that_ small? that seems off somehow, idk

In [19]:
frequencies[strain1]

0.01

In [20]:
frequencies[strain2]

0.02

oof, geez though, even of the small subset that might be relevant, less than 1% has the interaction
anyway that one probability is about twice as large as the other does kind of make intuitive sense? eh idk sort of maybe

In [21]:
strain1_non0_boolean = test_batch[:, strain1] >= 1

In [22]:
np.sum(strain1_non0_boolean)

207

In [23]:
strain2_non0_boolean = test_batch[:, strain2] >= 1

In [24]:
np.sum(strain2_non0_boolean)

377

In [25]:
double_strain_non0_boolean = strain1_non0_boolean | strain2_non0_boolean

In [26]:
np.sum(double_strain_non0_boolean)

577

In [27]:
double_strain_AND_non0_boolean = strain1_non0_boolean & strain2_non0_boolean

In [28]:
np.sum(double_strain_AND_non0_boolean)

7

In [29]:
not_double_strain_AND_non0_boolean = ~double_strain_AND_non0_boolean

In [30]:
strain1_non0_only_boolean = strain1_non0_boolean & not_double_strain_AND_non0_boolean

In [31]:
np.sum(strain1_non0_only_boolean)

200

checks out, good
taking the difference of such a small set seems like a waste though idk...

In [32]:
strain2_non0_only_boolean = strain2_non0_boolean & not_double_strain_AND_non0_boolean

In [33]:
np.sum(strain2_non0_only_boolean)

370

ok great everything checks out, great, fantastic, yay

In [34]:
np.where(double_strain_AND_non0_boolean)

(array([ 159, 1263, 6664, 6668, 7746, 8085, 9058]),)

In [35]:
all_indices_backup = np.where(double_strain_non0_boolean)
len(all_indices_backup)

1

In [36]:
all_indices_backup = np.where(double_strain_non0_boolean)[0]
len(all_indices_backup)

577

In [37]:
indices_both_backup = np.where(double_strain_AND_non0_boolean)[0]
len(indices_both_backup)

7

In [38]:
strain1_only_indices_backup = np.where(strain1_non0_only_boolean)[0]
len(strain1_only_indices_backup)

200

In [39]:
strain2_only_indices_backup = np.where(strain2_non0_only_boolean)[0]
len(strain2_only_indices_backup)

370

test set approach

In [40]:
strain1_non0_indices = np.where(strain1_non0_boolean)[0]
len(strain1_non0_indices)

207

In [41]:
strain2_non0_indices = np.where(strain2_non0_boolean)[0]
len(strain2_non0_indices)

377

In [42]:
strain1_non0_indices = set(strain1_non0_indices)
strain2_non0_indices = set(strain2_non0_indices)

In [43]:
%%timeit
strain1_non0_indices.intersection(strain2_non0_indices)

7.92 µs ± 839 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [44]:
%%timeit
strain1_non0_boolean & strain2_non0_boolean

5.7 µs ± 596 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


dang wow, good for you NumPy, that's legit impressive imo

In [45]:
strain1_non0_indices.intersection(strain2_non0_indices)

{159, 1263, 6664, 6668, 7746, 8085, 9058}

In [46]:
indices_both_backup

array([ 159, 1263, 6664, 6668, 7746, 8085, 9058])

In [47]:
indices_both_set = strain1_non0_indices.intersection(strain2_non0_indices)

In [48]:
%%timeit
strain1_non0_indices - indices_both_set
strain2_non0_indices - indices_both_set

33.9 µs ± 691 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [49]:
%%timeit
temp = ~double_strain_AND_non0_boolean
strain1_non0_boolean & temp
strain2_non0_boolean & temp

15.3 µs ± 2.37 µs per loop (mean ± std. dev. of 7 runs, 100000 loops each)


wow geez NumPy. yeah I vaguely remember testing/trying something like this with the log ratio coefficients, wanting to use the set approach, but that time too the NumPy Boolean vector appeared to prove faster.

to the extent we are willing to suffer the extensive waiting, we can look into what happens vis a vis time on the full problem size...

In [50]:
uber_strain1_non0_boolean = droplets[:,strain1] > 0
uber_strain2_non0_boolean = droplets[:,strain2] > 0

In [51]:
uber_strain1_non0_indices = set(np.where(uber_strain1_non0_boolean)[0])
uber_strain2_non0_indices = set(np.where(uber_strain2_non0_boolean)[0])

In [52]:
np.sum(uber_strain1_non0_boolean)

293822

In [53]:
len(uber_strain1_non0_indices)

293822

In [54]:
np.sum(uber_strain2_non0_boolean)

581664

In [55]:
len(uber_strain2_non0_indices)

581664

In [56]:
%%timeit
uber_strain1_non0_indices.intersection(uber_strain2_non0_indices)

41.3 ms ± 1.56 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [57]:
%%timeit
uber_strain1_non0_boolean & uber_strain2_non0_boolean

23.5 ms ± 1.75 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


dang wow, once again NumPy emerges victorious. so much for hashing saving us time I guess. of course I would need a better reason to explain the difference, since it would seem like algorithmically the set approach should be better than the other... (because you reduce the problem size by ignoring elements that are irrelevant)

In [58]:
uber_indices_both_strains = uber_strain1_non0_indices.intersection(uber_strain2_non0_indices)
uber_double_strain_AND_non0_boolean = uber_strain1_non0_boolean & uber_strain2_non0_boolean

In [59]:
%%timeit
uber_strain1_non0_indices - uber_indices_both_strains
uber_strain2_non0_indices - uber_indices_both_strains

212 ms ± 12.4 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [60]:
%%timeit
uber_temp = ~uber_double_strain_AND_non0_boolean
uber_strain1_non0_boolean & uber_temp
uber_strain2_non0_boolean & uber_temp

58.5 ms ± 3.22 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


welp, that's what I get for trying to beat NumPy I guess...
I mean I guess I shouldn't be complaining but I am still nevertheless genuinely surprised

In [61]:
uber_strain1_only_non0_indices = uber_strain1_non0_indices - uber_indices_both_strains
uber_strain2_only_non0_indices = uber_strain2_non0_indices - uber_indices_both_strains

In [62]:
uber_temp = ~uber_double_strain_AND_non0_boolean
uber_strain1_only_non0_boolean = uber_strain1_non0_boolean & uber_temp
uber_strain2_only_non0_boolean = uber_strain2_non0_boolean & uber_temp

In [63]:
len(uber_strain1_only_non0_indices)

282452

In [64]:
np.sum(uber_strain1_only_non0_boolean)

282452

In [65]:
len(uber_strain2_only_non0_indices)

570294

In [66]:
np.sum(uber_strain2_only_non0_boolean)

570294

ok, good sanity checks...

In [67]:
def get_observed_counts(strain1, strain2, counts):
    """Return observed counts for a given interaction."""
    assert counts.shape[1] == frequencies.size
    strain1_non0_boolean = counts[:,strain1] > 0
    strain2_non0_boolean = counts[:,strain2] > 0
    
    both_strains_non0_boolean = strain1_non0_boolean & strain2_non0_boolean
    not_both_strains_non0_boolean = ~both_strains_non0_boolean
    
    strain1_only_non0_boolean = strain1_non0_boolean & not_both_strains_non0_boolean
    strain2_only_non0_boolean = strain2_non0_boolean & not_both_strains_non0_boolean
    
    return (np.sum(strain1_only_non0_boolean), np.sum(both_strains_non0_boolean), np.sum(strain2_only_non0_boolean))

In [68]:
get_observed_counts(strain1, strain2, droplets)

(282452, 11370, 570294)

In [69]:
len(uber_indices_both_strains)

11370

OK nice, the function checks out, good.

In [70]:
get_observed_counts(strain1, strain2, test_batch)

(200, 7, 370)

nice/perfect/those are exactly the correct answers (afaict). OK great, so we have correct-functioning code for this it seems

In [71]:
def get_conditional_probabilities(strain1, strain2, single_strain_non0_probs, double_strain_non0_probs):
    strain1_non0_strain2_0_conditional_prob = 1. - (single_strain_non0_probs[strain2]/double_strain_non0_probs[strain1, strain2])
    strain2_non0_strain1_0_conditional_prob = 1. - (single_strain_non0_probs[strain1]/double_strain_non0_probs[strain1, strain2])
    both_non0_conditional_prob = 1. - strain1_non0_strain2_0_conditional_prob - strain2_non0_strain1_0_conditional_prob
    
    return np.asarray([strain1_non0_strain2_0_conditional_prob, both_non0_conditional_prob, 
                       strain2_non0_strain1_0_conditional_prob])

def get_conditional_expected_counts(strain1, strain2, single_strain_non0_probs, double_strain_non0_probs, total_number):
    
    conditional_probabilities = get_conditional_probabilities(strain1=strain1, strain2=strain2, 
                                                              single_strain_non0_probs=single_strain_non0_probs,
                                                              double_strain_non0_probs=double_strain_non0_probs)
    return total_number * conditional_probabilities

In [72]:
def get_single_and_double_strain_non0_probs(frequencies, rate=2.):
    frequencies = np.asarray(frequencies)
    single_strain_0_probs = np.exp(-(rate*frequencies))
    double_strain_0_probs = np.outer(single_strain_0_probs, single_strain_0_probs)

    single_strain_non0_probs = 1. - single_strain_0_probs
    double_strain_non0_probs = 1. - double_strain_0_probs
    return (single_strain_non0_probs, double_strain_non0_probs)

In [73]:
test1, test2 = get_single_and_double_strain_non0_probs(frequencies, rate=2)

In [74]:
test1[strain1]

0.019801326693244747

In [75]:
1. - np.exp(-2*frequencies[strain1])

0.019801326693244747

In [76]:
from scipy.stats import chi2

In [77]:
1. - np.exp(-2*(frequencies[strain1] + frequencies[strain2]))

0.05823546641575128

In [78]:
test2[strain1, strain2]

0.05823546641575139

In [79]:
test2[strain2, strain1]

0.05823546641575139

In [80]:
get_conditional_probabilities(strain1, strain2, test1, test2)

array([0.32668933, 0.01333244, 0.65997822])

In [81]:
get_conditional_probabilities(strain2, strain1, test1, test2)

array([0.65997822, 0.01333244, 0.32668933])

OK the above values are slightly more reasonable than before. (specifically with reference to the both non0 probability, which was less than 1% last time.) 

let's double check we get the numbers from before if the rate = 1


(also the rate being one would explain before why the probabilities were small)

In [82]:
get_conditional_probabilities(strain1, strain2, *get_single_and_double_strain_non0_probs(frequencies, rate=1))

array([0.33000561, 0.00666656, 0.66332783])

ok great, these are effectively the exact same numbers as before

and let me guess, the middle probability increases as the rate increases? which means that all of these hypothesis tests depend crucially on the assumed rate for the Poisson, which we don't _actually_ know?

oh no

In [83]:
get_conditional_probabilities(strain1, strain2, *get_single_and_double_strain_non0_probs(frequencies, rate=3))

array([0.32338482, 0.019997  , 0.65661818])

In [84]:
get_conditional_probabilities(strain1, strain2, *get_single_and_double_strain_non0_probs(frequencies, rate=4))

array([0.32009241, 0.02665956, 0.65324803])

yes this is true, although arguably the probabilities at least in this case don't seem to depend _too_ strongly on the assumed Poisson rate (but they definitely _do_ depend on it, oh no)

In [85]:
get_conditional_expected_counts(strain1, strain2, *get_single_and_double_strain_non0_probs(frequencies, rate=2),
                             np.sum(get_observed_counts(strain1, strain2, test_batch)))

array([188.49974403,   7.69282049, 380.80743548])

In [86]:
get_conditional_expected_counts(strain1, strain2, *get_single_and_double_strain_non0_probs(frequencies, rate=2),
                             np.sum(get_observed_counts(strain1, strain2, droplets)))

array([282297.47800931,  11520.77863012, 570297.74336057])

wow, the oberved counts line up with those for the CPM way more than would be expected, this is actually legit kind of crazy (crazy awesome haha). So hm, maybe the difference between the CPM and these over-dispersed methods is for the picky groups only, but not for the gluttonous groups?

this would be a questionable way to test fit, but you could like Holm-Bonferroni all of these and test fit based on that too maybe... not sure how to collapse all of that into a single statistic though...

In [87]:
def get_conditional_pairwise_pearson_categorical_divergence(strain1, strain2, single_strain_non0_probs, double_strain_non0_probs, counts):
    
    observed_counts = get_observed_counts(strain1=strain1, strain2=strain2, counts=counts)
    total_number = np.sum(observed_counts)
    
    if total_number == 0:
        return np.nan
    else:
        expected_counts = get_conditional_expected_counts(strain1=strain1, strain2=strain2, 
                                                          single_strain_non0_probs=single_strain_non0_probs,
                                                          double_strain_non0_probs=double_strain_non0_probs,
                                                          total_number=total_number)

        divergence = np.sum(((observed_counts - expected_counts)**2)/expected_counts, axis=None)
        return divergence

In [88]:
expected_counts = get_conditional_expected_counts(strain1, strain2, *get_single_and_double_strain_non0_probs(frequencies, rate=2),
                             np.sum(get_observed_counts(strain1, strain2, droplets)))

In [89]:
observed_counts = get_observed_counts(strain1, strain2, droplets)

In [90]:
np.sum(((observed_counts - expected_counts)**2)/expected_counts)

2.0579267994361095

In [91]:
get_conditional_pairwise_pearson_categorical_divergence(strain1, strain2,  *get_single_and_double_strain_non0_probs(frequencies, rate=2),
                                           droplets)

2.0579267994361095

OK, the code seems to work, that's good at least...

In [92]:
chi2.sf(2.0579267994361095, df=2)

0.35737722596057886

In [93]:
from scipy.stats import chisquare

In [94]:
chisquare(observed_counts, expected_counts)

Power_divergenceResult(statistic=2.0579267994361095, pvalue=0.35737722596057886)

OK good, good sanity check, seems I got the right answers, yay!

In [95]:
def get_all_conditional_pairwise_pearson_categorical_divergences_and_p_values(counts, frequencies, rate=2.):
    single_strain_non0_probs, double_strain_non0_probs = get_single_and_double_strain_non0_probs(frequencies=frequencies, rate=rate)
    
    divergences = np.zeros((frequencies.size, frequencies.size))
    pvals = np.zeros((frequencies.size, frequencies.size))
    
    for strain1 in range(frequencies.size):
        for strain2 in range(strain1+1, frequencies.size):
            divergence = get_conditional_pairwise_pearson_categorical_divergence(strain1=strain1, strain2=strain2, 
                            single_strain_non0_probs=single_strain_non0_probs, 
                            double_strain_non0_probs=double_strain_non0_probs, counts=counts)
            
            pval = chi2.sf(divergence, df=2)
            divergences[strain1, strain2] = divergence
            pvals[strain1, strain2] = pval
    return divergences, pvals

In [96]:
test = get_all_conditional_pairwise_pearson_categorical_divergences_and_p_values(test_batch, frequencies, rate=2)

In [97]:
np.where(np.isnan(test[0]))

(array([0, 0, 7]), array([7, 9, 9]))

In [98]:
test[0][0,7]

nan

In [99]:
test[0][0,9]

nan

In [100]:
test[0][7,9]

nan

In [101]:
get_observed_counts(0,7, droplets)

(2927, 1, 2938)

In [102]:
get_observed_counts(7,0, droplets)

(2938, 1, 2927)

what none of these are 0? how could any of these be zero??? ugh...

although admittedly returning NaN when the number of observations is 0 _is_ probably the correct answer. but still it doesn't seem to be happening here? what the heck????

In [103]:
get_conditional_expected_counts(0,7,  *get_single_and_double_strain_non0_probs(frequencies, rate=2), 2938+1+2927)

array([2.93270670e+03, 5.86599998e-01, 2.93270670e+03])

In [104]:
get_conditional_pairwise_pearson_categorical_divergence(0,7,  *get_single_and_double_strain_non0_probs(frequencies, rate=2), droplets)

0.3119977224264664

oh i'm dumb, it's because they were 0 for `test_batch`, right?

In [105]:
get_observed_counts(0,7, test_batch)

(0, 0, 0)

oof. ok that explains that 

In [106]:
get_observed_counts(0,9, test_batch)

(0, 0, 0)

In [107]:
get_observed_counts(7,9, test_batch)

(0, 0, 0)

basically there were zero observations of strains 0, 7, or 9, in `test_batch`. which hey makes sense given that this is only 10,000 droplets, and they are very low frequency

In [108]:
frequencies[0] * 10000

1.0

In [109]:
# uber_test = get_all_conditional_pairwise_pearson_categorical_divergences_and_p_values(droplets, frequencies, rate=2)

ok I am not sure what to do now, even when seemingly having the code be correct... like where to go further from here. 

Anyway assuming that 22 milliseconds is an accurate estimate for each interaction, and there are about 10000 interactions, then that is `10*22 = 220` nearly four minutes for this to run, and I think the 22 milliseconds per interaction figure is an underestimate (although I'm not sure what the correct value is). Anyway this should at the very least make me glad I didn't insist on trying to use the set-based version, given that would have been more than twice as slow as this (which fyi is pretty darn slow...)

I guess at the very least I could (or "should"?) use this to double-check the conclusion from the picky-groups test that the counts do not resemble those from a CPM distribution.

and then we also "need" to use it on counts generated from the interaction-dirichlet distribution (to try to show that these pairwise tests could be used to detect interactions from censoring effects)

in which case we would also need to modify the above code to also return the sign vector corresponding to `observed - expected` for every interaction -- which seems like it would be difficult or at least ugly to do. well yeah you just need to make a third order tensor/array, third mode stores the signs, first two just correspond to the entries of the p value matrix essentially, right? so yeah

In [110]:
# uber_test[0]

In [111]:
# uber_test[1]

OK that's not great, even though there weren't any interactions, some of these are getting rejected, seemingly mostly for the high frequency strains. sort of not surprising, since more data, more power to reject, at least ostensibly. and it's not rejecting the null because there are interactions, but because the ratios are not characteristic of the CPM distribution (as opposed to a more over-dispersed variant).

In [112]:
# np.savez_compressed('conditional_pairwise_tests_demo.npz', divergences=uber_test[0], chi2_approx_pvals=uber_test[1])

In [113]:
# npzfile = np.load('conditional_pairwise_tests_demo.npz')

In [114]:
# assert np.all(uber_test[0] == npzfile['divergences'])
# assert np.all(uber_test[1] == npzfile['chi2_approx_pvals'])

ok good, so I never have to run the above cell again -- will now comment it out

In [115]:
def get_conditional_pairwise_pearson_categorical_divergence(strain1, strain2, single_strain_non0_probs, double_strain_non0_probs, counts, signs=False):
    
    observed_counts = get_observed_counts(strain1=strain1, strain2=strain2, counts=counts)
    total_number = np.sum(observed_counts)
    
    if total_number == 0:
        return np.nan if signs == False else (np.nan, np.array([np.nan, np.nan, np.nan]))
    else:
        expected_counts = get_conditional_expected_counts(strain1=strain1, strain2=strain2, 
                                                          single_strain_non0_probs=single_strain_non0_probs,
                                                          double_strain_non0_probs=double_strain_non0_probs,
                                                          total_number=total_number)

        divergence = np.sum(((observed_counts - expected_counts)**2)/expected_counts, axis=None)
        return divergence if signs == False else (divergence, np.sign(observed_counts - expected_counts))
    
def get_all_conditional_pairwise_pearson_categorical_divergences_and_p_values(counts, frequencies, rate=2., signs=False):
    single_strain_non0_probs, double_strain_non0_probs = get_single_and_double_strain_non0_probs(frequencies=frequencies, rate=rate)
    
    divergences = np.zeros((frequencies.size, frequencies.size))
    pvals = np.zeros((frequencies.size, frequencies.size))
    if signs == True:
        sign_vectors = np.zeros((frequencies.size, frequencies.size, 3))
    
    for strain1 in range(frequencies.size):
        for strain2 in range(strain1+1, frequencies.size):
            result = get_conditional_pairwise_pearson_categorical_divergence(strain1=strain1, strain2=strain2, 
                            single_strain_non0_probs=single_strain_non0_probs, 
                            double_strain_non0_probs=double_strain_non0_probs, counts=counts, signs=signs)
            if signs == False:
                divergence = result
            else:
                divergence, sign_vector = result
            
            pval = chi2.sf(divergence, df=2)
            divergences[strain1, strain2] = divergence
            pvals[strain1, strain2] = pval
            
            if signs == True:
                sign_vectors[strain1, strain2, :] = sign_vector
    return (divergences, pvals) if signs == False else (divergences, pvals, sign_vectors)

In [116]:
newtest1, newtest2 = get_all_conditional_pairwise_pearson_categorical_divergences_and_p_values(test_batch, frequencies, rate=2)

In [117]:
np.testing.assert_array_equal(newtest1, test[0])
np.testing.assert_array_equal(newtest2, test[1])

looking good so far

In [118]:
newtest3, newtest4, newtest5 = get_all_conditional_pairwise_pearson_categorical_divergences_and_p_values(test_batch, frequencies, rate=2, signs=True)

In [119]:
np.testing.assert_array_equal(newtest3, test[0])
np.testing.assert_array_equal(newtest4, test[1])

In [120]:
newtest5[0,7,:]

array([nan, nan, nan])

In [121]:
newtest5[0,9,:]

array([nan, nan, nan])

In [122]:
newtest5[7,9,:]

array([nan, nan, nan])

In [123]:
newtest5

array([[[ 0.,  0.,  0.],
        [-1., -1.,  1.],
        [-1., -1.,  1.],
        ...,
        [-1., -1.,  1.],
        [-1., -1.,  1.],
        [-1., -1.,  1.]],

       [[ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        [-1., -1.,  1.],
        ...,
        [-1.,  1.,  1.],
        [-1., -1.,  1.],
        [-1., -1.,  1.]],

       [[ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        ...,
        [ 1., -1.,  1.],
        [ 1., -1.,  1.],
        [ 1., -1., -1.]],

       ...,

       [[ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        ...,
        [ 0.,  0.,  0.],
        [ 1., -1., -1.],
        [ 1.,  1., -1.]],

       [[ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        ...,
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 1.,  1., -1.]],

       [[ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        ...,
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.]]

OK sweet, looks good to me.

In [124]:
from simulations.local_chi2_cpm import get_all_conditional_pairwise_pearson_categorical_divergences_and_p_values as external_version

In [125]:
newtest6, newtest7, newtest8 = external_version(test_batch, frequencies, rate=2, signs=True)

In [126]:
np.testing.assert_array_equal(newtest6, test[0])
np.testing.assert_array_equal(newtest7, test[1])
np.testing.assert_array_equal(newtest5, newtest8)